<span id="tocheading" style='font-size: 24px; font-weight: bold'>Table of Contents</span>
<div id="toc"></div>

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sea
sea.set_context("talk", font_scale=1.5)
sea.set_style("darkgrid")
#plt.style.use('ggplot')
%matplotlib inline

# JB Experiment Results

In [ ]:
# Read JB file.
jb_file = r"jb_experiment.tsv"
df_jb = pd.read_csv(jb_file, sep="\t")

In [ ]:
# Date as index.
df_jb['datetime'] = pd.to_datetime(df_jb['date'])
df_jb = df_jb.set_index('datetime')
df_jb.drop(['date'], axis=1, inplace=True)

In [ ]:
df_jb.columns

In [ ]:
# Compute Error bars.
df_jb["Control TAP SE"] = df_jb["Control TAP"] - df_jb["Control TAP Lower"]
df_jb["Treatment TAP SE"] = df_jb["Treatment TAP"] - df_jb["Treatment TAP Lower"]

In [ ]:
fig, ax = plt.subplots(figsize=[12, 6])

# Use primary axis for relative improvements
ax.plot(df_jb.index.values, 100*df_jb["TAP Change"], '--', color='black')

for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    
ax.set_ylim([-1.2, 0])
ax.set_ylabel("TAP % Reduction", fontsize=22, weight = 'bold')

# Use secondary axis for absolute improvements
ax2 = ax.twinx()
ax2.errorbar(df_jb.index.values, df_jb["Control TAP"], yerr=df_jb["Control TAP SE"], linestyle='-.', color='0.5')
ax2.errorbar(df_jb.index.values, df_jb["Treatment TAP"], yerr=df_jb["Treatment TAP SE"], color='0.2')
ax2.set_ylim([0.118, 0.128])
ax2.set_yticklabels([])
ax2.set_ylabel("TAP Absolute (masked)", fontsize=20, weight = 'bold')

# Put Legends
ax.legend(["% Reduction"], loc='upper left')
ax2.legend(["Control", "Treatment"])

fig.savefig("jb_experiment.pdf", bbox_inches='tight')

# VW Experiments

## Interaction Terms

In [ ]:
df_interactions = pd.read_csv("vw_interactions.tsv", sep="\t")
df_interactions.columns

In [ ]:
# VW Interactions
sea.set_context("poster", font_scale=1.3)
fig, ax = plt.subplots(figsize=[12, 8])

# Use primary axis for relative improvements
ax.plot(df_interactions["Interactions"], 100*df_interactions["Improvements (%)"], '-o', color='black')

for tick in ax.get_xticklabels():
    tick.set_rotation(45)
    
ax.set_ylim([0, 3])
ax.set_ylabel("Improvements [%]")

# Use secondary axis
ax2 = ax.twinx()
ax2.plot(df_interactions["Interactions"], df_interactions["Model Size (KB)"], '--x', color='black')
ax2.set_ylabel("Model Size [KB]")
ax2.set_ylim([0, 600])

# Put Legends
ax.legend(["Improvements [%]"], loc='upper left')
ax2.legend(["Model Size [KB]"], loc="upper right")

# fig.savefig("vw_interactions.pdf", bbox_inches='tight')

## Numbits


In [ ]:
df_numbits = pd.read_csv("vw_numbits.tsv", sep="\t")
df_numbits.columns

In [ ]:
# VW Numbits
fig, ax = plt.subplots(figsize=[12, 8])

# Use primary axis for relative improvements
ax.plot(df_numbits["Number of Bits"], 100*df_numbits["Improvements (%)"], '-o', color='black')

for tick in ax.get_xticklabels():
    tick.set_rotation(45)

ax.set_yticks(np.arange(0, 2.1, 0.4)) #[0, 0.4, 0.8, 1.2, 1.6, 2])
ax.set_ylim([0, 2])
ax.set_ylabel("Improvements [%]")

# Use secondary axis
ax2 = ax.twinx()
ax2.plot(df_numbits["Number of Bits"], df_numbits["Model Size (KB)"], '--x', color='black')
ax2.set_ylabel("Model Size [KB]")
ax2.set_ylim([0, 25])

# Put Legends
ax.legend(["Improvements [%]"], loc='lower left')
ax2.legend(["Model Size [KB]"], loc="lower right")

fig.savefig("vw_numbits.pdf", bbox_inches='tight')

## Exploration Strategy

In [ ]:
df_exploration = pd.read_csv("vw_all_regrets.csv")
df_exploration.columns
df_exploration = df_exploration.rolling(15, min_periods=1).mean()
df_exploration.set_index('n_iteration', inplace=True)

In [ ]:
# Plot exploration
fig, ax = plt.subplots(figsize=[12, 7.5])

explore_cols = ['--epsilon 0.1', '--epsilon 0.2',
                '--bag 5 --epsilon 0.05', 
                '--softmax --lambda 30 --epsilon 0.05']
greyscale = True

lines = ["--", "-", "-.", ":"]
for i, c in enumerate(explore_cols):
    plot_data = df_exploration[[c]]
    if greyscale:
        c = "{0}".format(round(0.5/(i+1), 1))
    else:
        c = None
    plot_data.plot(label=c, ax=ax, style=lines[i%len(lines)], color=c)
            
ax.set_ylim([0, 0.4])
ax.set_xlabel("Number of batches (batch size=20)")
ax.set_ylabel("Average Regret")


# Put Legends
ax.legend(explore_cols, loc='upper left', bbox_to_anchor= (0.0, 1.01), ncol=2, 
            borderaxespad=0, frameon=False, fontsize=22)

fig.savefig("vw_exploration.pdf", bbox_inches='tight')